In [111]:
#Code Below

In [112]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

In [113]:
is_valid=False

In [114]:
#Change this for validation with 20% of data

In [115]:
def feature_creation(df):
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow'] = df['datetime'].dt.dayofweek
    df['doy'] = df['datetime'].dt.dayofyear
    df.drop(['click_time','datetime'],axis=1,inplace=True)
    return df

In [116]:
start_time = time.time()
start_time

1525721785.32658

In [117]:
# train = pd.read_csv('./input/mnt/ssd/kaggle-talkingdata2/competition_files/train.csv')
# train.columns

In [118]:
train_columns = ['ip','app','device','os','channel','click_time','is_attributed']
test_columns = ['ip','app','device','os','channel','click_time','click_id']

In [119]:
dtypes = {
    'ip':'uint32',
    'app':'uint16',
    'device':'uint16',
    'os':'uint16',
    'channel':'uint16',
    'is_attribued':'uint8',
    'click_id':'uint32'
}

In [120]:
train = pd.read_csv('./input/mnt/ssd/kaggle-talkingdata2/competition_files/train.csv',skiprows=(1,123903891),nrows=51000000,usecols=train_columns,dtype=dtypes)
train.head(n=5)
#nrow_train = train.shape[0]

,ip,app,device,os,channel,click_time,is_attributed
0,17357,3,1,19,379,2017-11-06 14:33:34,0
1,35810,3,1,13,379,2017-11-06 14:34:12,0
2,45745,14,1,13,478,2017-11-06 14:34:52,0
3,161007,3,1,13,379,2017-11-06 14:35:08,0
4,18787,3,1,16,379,2017-11-06 14:36:26,0


In [121]:
test = pd.read_csv('test.csv',usecols=test_columns,dtype=dtypes)
print(test.head(n=5))

   click_id      ip  app  device  os  channel           click_time
0         0    5744    9       1   3      107  2017-11-10 04:00:00
1         1  119901    9       1   3      466  2017-11-10 04:00:00
2         2   72287   21       1  19      128  2017-11-10 04:00:00
3         3   78477   15       1  13      111  2017-11-10 04:00:00
4         4  123080   12       1  13      328  2017-11-10 04:00:00


In [122]:
print('[{}] Finished loading data'.format(time.time() - start_time))

[58.36669683456421] Finished loading data


In [123]:
y=train['is_attributed']
train.drop(['is_attributed'],axis=1,inplace=True)

In [124]:
sub=pd.DataFrame()
test.drop(['click_id'],axis=1,inplace=True)
gc.collect()

266

In [125]:
nrow_train = train.shape[0]
merge = pd.concat([train,test])

In [126]:
del train,test
gc.collect()

36

In [127]:
# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip','clicks_by_ip']
merge = pd.merge(merge,ip_count,on='ip',how='left',sort=False)
merge.head(n=5)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
merge.drop('ip',axis=1,inplace=True)
train = merge[:nrow_train]
test = merge[nrow_train:]

In [128]:
test.head(n=5)

,app,device,os,channel,click_time,clicks_by_ip
51000000,9,1,3,107,2017-11-10 04:00:00,377
51000001,9,1,3,466,2017-11-10 04:00:00,7621
51000002,21,1,19,128,2017-11-10 04:00:00,5877
51000003,15,1,13,111,2017-11-10 04:00:00,4600
51000004,12,1,13,328,2017-11-10 04:00:00,884


In [129]:
del test,merge
gc.collect()

202

In [130]:
train = feature_creation(train)
train.head(n=30)

,app,device,os,channel,clicks_by_ip,dow,doy
0,3,1,19,379,8808,0,310
1,3,1,13,379,3076,0,310
2,14,1,13,478,60724,0,310
3,3,1,13,379,527,0,310
4,3,1,16,379,1371,0,310
5,3,1,23,379,1988,0,310
6,3,1,19,379,1491,0,310
7,3,1,13,379,819,0,310
8,64,1,22,459,1233,0,310
9,3,1,25,379,9494,0,310


In [131]:
print('[{}] Start to generate time features'.format(time.time() - start_time))

[123.19655585289001] Start to generate time features


In [132]:
# #path = '/input/mnt/ssd/kaggle-talkingdata2/competition-files/'
# #train = pd.read_csv(path+"train.csv")
# train = pd.read_csv('./input/mnt/ssd/kaggle-talkingdata2/competition_files/train.csv')
# print(train.size)

In [133]:
# Setting the parameters for xgboost model
params = { 'eta':0.3,
           'tree_method':'approx',
           'grow_policy':'lossguide',
           'max_leaves':1600,
           'max_depth':8,
           'subsample':0.9,
           'colsample_bytree':0.7,
           'colsample_bylevel':0.7,
           'min_child_weight':0,
           'alpha':4,
           'objective':'binary:logistic',
           'scale_pos_weight':9,
           'eval_metric':'auc',
           'nthread':8,
           'random_state':99,
           'silent':True}

if (is_valid == True):
    x1,x2,y1,y2 = train_test_split(train,y,test_size=0.1,random_state=99)
    dtrain = xgb.DMatrix(x1,y1)
    dvalid = xgb.DMatrix(x2,y2)
    del x1,x2,y1,y2
    gc.collect()
    watchlist = [(dtrain,'train'),(dvalid,'valid')]
    model = xgb.train(params,dtrain,200,watchlist,maximize=True,early_stopping_rounds = 35,verbose_eval=5)
    del dvalid
    
else:
    dtrain = xgb.DMatrix(train,y)
    del train,y
    gc.collect()
    watchlist = [(dtrain,'train')]
    model = xgb.train(params,dtrain,30,watchlist,maximize=True,verbose_eval=1)
    
print('[{}] Finish XGBoost training'.format(time.time() - start_time))

[0]	train-auc:0.930386
[1]	train-auc:0.960488
[2]	train-auc:0.960788
[3]	train-auc:0.961054
[4]	train-auc:0.961737
[5]	train-auc:0.961022
[6]	train-auc:0.961166
[7]	train-auc:0.961228
[8]	train-auc:0.96407
[9]	train-auc:0.964753
[10]	train-auc:0.966301
[11]	train-auc:0.967606
[12]	train-auc:0.968483
[13]	train-auc:0.968478
[14]	train-auc:0.969442
[15]	train-auc:0.969026
[16]	train-auc:0.970004
[17]	train-auc:0.969745
[18]	train-auc:0.970297
[19]	train-auc:0.970145
[20]	train-auc:0.971084
[21]	train-auc:0.971328
[22]	train-auc:0.971315
[23]	train-auc:0.971547
[24]	train-auc:0.971662
[25]	train-auc:0.971764
[26]	train-auc:0.972329
[27]	train-auc:0.972767
[28]	train-auc:0.973166
[29]	train-auc:0.973243
[1098.2760050296783] Finish XGBoost training


In [134]:
test = pd.read_csv('test.csv',usecols=test_columns,dtype=dtypes)
test = pd.merge(test,ip_count,on='ip',how='left',sort=False)
test.head(n=5)

,click_id,ip,app,device,os,channel,click_time,clicks_by_ip
0,0,5744,9,1,3,107,2017-11-10 04:00:00,377
1,1,119901,9,1,3,466,2017-11-10 04:00:00,7621
2,2,72287,21,1,19,128,2017-11-10 04:00:00,5877
3,3,78477,15,1,13,111,2017-11-10 04:00:00,4600
4,4,123080,12,1,13,328,2017-11-10 04:00:00,884


In [135]:
sub['click_id'] = test['click_id'].astype('int')
test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')
test = feature_creation(test)
test.drop(['click_id','ip'],axis=1,inplace=True)
#test.drop(['ip'],axis=1,inplace=True)
print(test.columns)
dtest = xgb.DMatrix(test)
del test
gc.collect()

# Saving the Predictions
sub['is_attributed'] = model.predict(dtest,ntree_limit=model.best_ntree_limit)
sub.to_csv('final_sub.csv',float_format='%.8f',index=False)

Index(['app', 'device', 'os', 'channel', 'clicks_by_ip', 'dow', 'doy'], dtype='object')
